In [1]:
from rdkit import rdBase, Chem, DataStructs
from rdkit.Avalon import pyAvalonTools
from rdkit.Chem import AllChem
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit.Chem.AtomPairs import Pairs, Torsions
print(rdBase.rdkitVersion) # 2020.09.3

2023.03.2


In [2]:
import pandas as pd
df = pd.read_csv("/workspace/tamokuteki/DILIrank-DILIscore_List.csv")

In [3]:
import pubchempy as pcp

In [4]:
from rdkit.Chem import Crippen

In [5]:
from rdkit.Chem import Descriptors

In [ ]:
tsv = []
midashi  = ["name", 'MolecularFormula', 'MolecularWeight', 'CanonicalSMILES', 'IUPACName', 'LogP', 'TPSA', "fp"]

for i in range(len(df)):
    col = []
    name = df.iloc[i,1]
    sev = df.iloc[i,2]
    properties = ['MolecularFormula', 'MolecularWeight', 'CanonicalSMILES', 'IUPACName']
    k = pcp.get_properties(properties, name, 'name')
    try:
        result = k[0]
        if len(result) != 5:
            continue
    except:
        continue
    smiles = result["CanonicalSMILES"]
    mol = Chem.MolFromSmiles(smiles)
    
    metal = 0
    #filter 金属原子をのぞく
    for atom in mol.GetAtoms():
        if atom.GetAtomicNum() in [1,2,5,6,7,8,9,10,14,15,16,17,18,33,34,35,36,52,53,54,85]:
            continue
        else:
            metal += 100
    
    if metal >= 100:
        print(smiles)
        continue


    logp = Crippen.MolLogP(mol)
    tpsa = Descriptors.TPSA(mol)
    a = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=2048)
    col.append(name)
    col.append(sev)
    for n in result.keys():
        col.append(result[n])
    col.append(logp)
    col.append(tpsa)
    col.append(a)
    tsv.append(col)
    
    del smiles
    del result
    del mol
    del logp
    del tpsa
    del a

In [ ]:
midashi  = ["name","severity", "CID", 'MolecularFormula', 'MolecularWeight', 'CanonicalSMILES', 'IUPACName', 'LogP', 'TPSA', "fp"]

In [ ]:
pd.DataFrame(tsv).to_csv('/workspace/tamokuteki/231103_use.tsv', sep="\t", header=midashi, index=False)